In [7]:
from keras import layers
from archive.lib.models.graph import Graph

model = Graph(
    inputs=[
        layers.Input(shape=(784,), name='input1'),
        layers.Input(shape=(10,), name='input2')
    ],
    layers=[
        layers.Dense(128, activation='relu', name='dense1'),
        layers.Dense(64, activation='relu', name='dense2'),
        layers.Dense(64, activation='relu', name='dense3'),
        layers.Dense(10, activation='softmax', name='output')
    ],
    connections=[
        ('input1', 'dense1'),
        ('dense1', 'dense2'),
        ('dense2', 'output')
    ]
)

In [8]:
model.summary()

Model: "graph_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input1 (InputLayer)         [(None, 784)]             0         
                                                                 
 dense1 (Dense)              (None, 128)               100480    
                                                                 
 dense2 (Dense)              (None, 64)                8256      
                                                                 
 output (Dense)              (None, 10)                650       
                                                                 
Total params: 109386 (427.29 KB)
Trainable params: 109386 (427.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
model.summary()

Model: "graph_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input1 (InputLayer)         [(None, 784)]             0         
                                                                 
 dense1 (Dense)              (None, 128)               100480    
                                                                 
 dense2 (Dense)              (None, 64)                8256      
                                                                 
 output (Dense)              (None, 10)                650       
                                                                 
Total params: 109386 (427.29 KB)
Trainable params: 109386 (427.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [1]:
from archive.lib.models.graph import Connection, Graph
from tensorflow.keras import layers

# Assuming the Graph class is defined as previously discussed
num_encoder_tokens = 128
num_decoder_tokens = 128

latent_dim = 64

# Define your seq2seq architecture using the Graph model
seq2seq_model = Graph(
    inputs=[
        layers.Input(shape=(None, num_encoder_tokens), name='encoder_input'),
        layers.Input(shape=(None, num_decoder_tokens), name='decoder_input')
    ],
    layers=[
        layers.GRU(latent_dim, return_state=True, name='encoder_rnn'),
        layers.GRU(latent_dim, return_sequences=True, name='decoder_rnn'),
        layers.Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
    ],
    connections=[
        Connection('encoder_input', 'encoder_rnn'),
        # decoder_rnn accepts 2 named arguments.
        # The 'inputs' argument takes its value from the decoder_input. 
        # The 'initial_state' argument takes its value from the second output(index 1) of the encoder_rnn
        Connection('decoder_input', ('decoder_rnn', 'inputs')),
        Connection(('encoder_rnn', 1), ('decoder_rnn', 'initial_state')),
        Connection('decoder_rnn', 'decoder_dense'),
    ]
)

# Note: This simplified setup does not directly handle the state transfer between encoder and decoder.
# In practice, you would need a custom layer or mechanism to manage the LSTM states.


In [3]:
seq2seq_model.summary()

Model: "graph"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input (InputLayer)  [(None, None, 128)]          0         []                            
                                                                                                  
 decoder_input (InputLayer)  [(None, None, 128)]          0         []                            
                                                                                                  
 encoder_rnn (GRU)           [(None, 64),                 37248     ['encoder_input[0][0]']       
                              (None, 64)]                                                         
                                                                                                  
 decoder_rnn (GRU)           (None, None, 64)             37248     ['decoder_input[0][0]',   